# Lab 2: Primer to NGS Data Analysis
___

## Table of Contents

- [1. Introduction to NGS Data Analysis](#1.-Introduction) 
- [2. Genome and Simulated Read Data](#2.-Genome-and-Simulated-Read-Data)
- [3. Short read alignment](#3.-Read-alignment)
- [4. samtools:-Manipulating-the-alignment-files](#4.-samtools:-Manipulating-the-alignment-files)
- [5. Understanding the VCF Files](#5.-Understanding-the-VCF-Format)
- [6. Visualizing the alignment](#6.-Visualing-the-mapping-of-reads)
- [7. Further reading](#7.-References)
- [8. Exercises]

### Prerequisite installation

1. [wgsim](https://github.com/lh3/wgsim): Reads simulator.
2. [bwa](https://github.com/lh3/bwa): short-read alignment tools.
3. [samtools](https://samtools.sourceforge.net): SAM manipulating tools.
4. [bcftools](https://github.com/samtools/bcftools): VCF file manipulator.

## 1. Introduction

Next-generation sequencing (NGS) refers to the emerging high-throughput technologies for sequencing DNA or RNA. A typical workflow for NGS data analysis is usually composed of the following steps:

![](../images/ngs_overview.png)

`bwa` and `samtools` are open-source tools for NGS data analyis, which fit into step3, step 4-5 respectively.

[BWA](http://bio-bwa.sourceforge.net/) is probably the most widely used short read aligner ever. By now, it’s even part of the official Illumina analysis pipeline. BWA actually contains three different aligner algorithms:
* BWA-Backtrack: used for short-read alignment using FM-Index strategy.
* BWA-SW: used for long reads alignment, utilizing seed (FM-index) and extend (smith-waterman) strategy.
* BWA-Mem: used for long-read alignment.


As a companion tools, `samtools` can process aligned sequence reads, and manipulate them with ease:
* convert between the two most common file formats (SAM and BAM), 
* sort and index files (for speedy retrieval later),
* extract specific genomic regions of interest.

`samtools` also enables quality checking of reads, and automatic identification of genomic variants.

## 2. Genome and Simulated Read Data

For simplicity, we take a small set of simulated reads from *E. coli*. We use E. coli because its genome is quite small — 4,649 kilobases, with 4,405 genes, and its entire genome fits into a single FASTA file of 4.8 megabytes.

### 2.1 Genome files

You can download directly from NCBI ftp to get the genome sequence file for E.coli.

### 2.2 Generating simulated reads

Here we use `[wgsim](https://github.com/lh3/wgsim)` tools to generate a small set of artificial reads data.
```bash
Program: wgsim (short read simulator)
Version: 0.3.0
Contact: Heng Li <lh3@sanger.ac.uk>

Usage:   wgsim [options] <in.ref.fa> <out.read1.fq> <out.read2.fq>

Options: -e FLOAT      base error rate [0.020]
         -d INT        outer distance between the two ends [500]
         -s INT        standard deviation [50]
         -N INT        number of read pairs [1000000]
         -1 INT        length of the first read [70]
         -2 INT        length of the second read [70]
         -r FLOAT      rate of mutations [0.0010]
         -R FLOAT      fraction of indels [0.15]
         -X FLOAT      probability an indel is extended [0.30]
         -S INT        seed for random generator [-1]
         -h            haplotype mode
```

By default, `wgsim` will simulate $10^6$ paired-end reads. Here you need to generate $10^4$ single-end reads of length 50 by setting `out.read2.fq` to be `/dev/null`, while setting point mutation rate, indels fraction and the other options as default:
```bash
mkdir simulated_reads
wgsim -N 10000 -d 300 -1 50 -2 50 -S 1234 genomes/NC_008253.fna simulated_reads/sim_reads.fq /dev/null
```

## 3. Read alignment

The next step is to align the artificial reads to the reference genome. Here we use `bwa` to perform the task.

### 3.1 Building genome index

The first step for read alignment is to build the FM-index for the genome string. For `bwa`, the command is `bwa index`:
```bash
Usage:   bwa index [options] <in.fasta>

Options: -a STR    BWT construction algorithm: bwtsw or is [auto]
         -p STR    prefix of the index [same as fasta name]
         -b INT    block size for the bwtsw algorithm (effective with -a bwtsw) [10000000]
         -6        index files named as <in.fasta>.64.* instead of <in.fasta>.* 

Warning: `-a bwtsw' does not work for short genomes, while `-a is' and
         `-a div' do not work not for long genomes.

```

Since E.coli is a small genome, we need to choose the `is` algorithm to build the genome index:
```bash
mkdir indexes
bwa index -a is -p indexes/e_coli genomes/NC_008253.fna
```
This command will generate 5 files under the `indexes` directory:
* `e_coli.amb`: summary of the ambiguous character in the genome sequence
* `e_coli.ann`: annotation of the genome sequence
* `e_coli.bwt`: Burrow-wheeler transform (BWT) of the genome
* `e_coli.pac`: packed format of the genome string
* `e_coli.sa`: sampled suffix array

### 3.2 Align the short reads to the genome index

`bwa` can perform 3 algorithms for read mapping：
- `bwa-backtrack`: used for alignment of short reads shorter than 70bp;
- `bwa-sw`: hybrid of Smith-Waterman algorithm and FM-index, for alignment of longer reads;
- `bwa-mem`: memory-efficient algorithm for alignment of longer reads.

Here we use `bwa-backtrack` algorithm (`bwa aln` + `bwa samse`) to conduct the alignment. Here is the help information for `bwa aln`:
```bash
Usage:   bwa aln [options] <prefix> <in.fq>

Options: -n NUM    max #diff (int) or missing prob under 0.02 err rate (float) [0.04]
         -o INT    maximum number or fraction of gap opens [1]
         -e INT    maximum number of gap extensions, -1 for disabling long gaps [-1]
         -i INT    do not put an indel within INT bp towards the ends [5]
         -d INT    maximum occurrences for extending a long deletion [10]
         -l INT    seed length [32]
         -k INT    maximum differences in the seed [2]
         -m INT    maximum entries in the queue [2000000]
         -t INT    number of threads [1]
         -M INT    mismatch penalty [3]
         -O INT    gap open penalty [11]
         -E INT    gap extension penalty [4]
         -R INT    stop searching when there are >INT equally best hits [30]
         -q INT    quality threshold for read trimming down to 35bp [0]
         -f FILE   file to write output to instead of stdout
         -B INT    length of barcode
         -L        log-scaled gap penalty for long deletions
         -N        non-iterative mode: search for all n-difference hits (slooow)
         -I        the input is in the Illumina 1.3+ FASTQ-like format
         -b        the input read file is in the BAM format
         -0        use single-end reads only (effective with -b)
         -1        use the 1st read in a pair (effective with -b)
         -2        use the 2nd read in a pair (effective with -b)
         -Y        filter Casava-filtered sequences
```
and `bwa samse` for single-end reads:
```bash
bwa samse [-n max_occ] [-f out.sam] [-r RG_line] <prefix> <in.sai> <in.fq>
```

Therefore, we run
```bash
bwa aln indexes/e_coli simulated_reads/sim_reads.fq -f alignments/sim_reads_mapped.sai
bwa samse -f alignments/sim_reads_mapped.sam indexes/e_coli alignments/sim_reads_mapped.sai simulated_reads/sim_reads.fq
```

Have a look at the **sam index** file `*.sai` and the SAM file `*.sam`.

### 3.3 Interpret the SAM output

![](../images/samtools1.png)
As SAMtools is primarily concerned with manipulating SAM files, it is useful to take a look at the sample SAM file generated by `bwa`, and to dive into the details of the SAM file format.

A typical SAM file contains two types of lines:
- **header** lines starting with a `@`, which provide the meta-data required for the alignment;
- **alignment** lines starting without `@`, each line of which describe the single alignment information of each read against the reference genome. 

Let's start with the first alignment line for illustrative purpose, which contains 11 mandatory fields, followed by a number of user-defined optional fields.

#### (1) Mandatory fields

The mandatory fields include the following information:

| Col | Field | Type | Description | Example |
| --- | --- | --- | --- | --- |
| 1 |  QNAME | str | query name of the read or the read pair | `NC_008253.1_0` |
| 2 | FLAG | int | bitwise flags (pairing, mapped, mate mapped, etc.) | 16: indicates that the read maps to the reverse strand of the genome. | 
| 3 | RNAME | str | reference sequence name |  `NC_008253.1` |
| 4 | POS | int | 1-based leftmost position of clipped alignment | `407` |
| 5 | MAPQ | int | PHRED-scale mapping quality | `37` |
| 6 | CIGAR | str | extended CIGAR string for alignment details | `70M` |
| 7 | RNEXT | str | mate reference name ('=' for the same) |  * |
| 8 | PNEXT | int | position of mate/next segment | 0 |
| 9 | TLEN | int | observed template length | 0 |
| 10 | SEQ | str | segment sequence | `CCAGGCAGTGGCAGGTGGCC...T` |
| 11 | QUAL | str | ASCII of PHRED-scale base quality | `22222222222222222222...2` | 

#### (2) `FLAG` field
Each bits in the `FLAG` field is defined as

| Chr |	Flag |	Description |
| --- | --- | --- |
| p | `0x0001` | the read is paired in sequencing |
| P	| `0x0002` | the read is mapped in a proper pair |
| u	| `0x0004` | the query sequence itself is unmapped |
| U | `0x0008` |  the mate is unmapped |
| r | `0x0010` |  strand of the query (1 for reverse) |
| R | `0x0020` | strand of the mate |
| 1 | `0x0040` | the read is the first read in a pair |
| 2 | `0x0080`| the read is the second read in a pair |
| s	| `0x0100` | the alignment is not primary |
| f | `0x0200` | QC failure |
| d | `0x0400`|	optical or PCR duplicate |


#### (3) Optional fields

The optional SAM fields always use the format: **FIELD_NAME:DATA_TYPE:VALUE**. Here is a list of  the possible **DATA_TYPEs**
- `A`: **character**
- `f`: **float**
- `H`: **hexadecimal string**
- `i`: **integer**
- `Z`: **string**


| Fields | Description | Data Type |
| --- | --- |  --- |
| `XT:A:U` |   The mapping `Type` is `Unique`        |  Character: Unique/Repeat/N/Mate-sw | 
| `NM:i:1` | 	The edit distance to the reference is 1 | Integer | 
| `X0:i:1` |  Number of best-hit is 1 | Integer |
|`X1:i:0` |	 Number of sub-optimal hits is 0 | Integer |
| `XM:i:1` |  Number of the mismatches in the alignment is 1 | Integer |
|`XO:i:0` |	 Number of gap-Open is 0 | Integer | 
| `XG:i:0` | Number of Gap-extensions is 0 | Integer | 
| `MD:Z:8G61` |  8 matches followed by a mismatch to `G` in reference and then 61 matches | String: Mismatch positions and bases |


For further about SAM/BAM format, please refer to https://samtools.github.io/hts-specs/SAMv1.pdf. 

For FLAG information, you can refer to http://picard.sourceforge.net/explain-flags.html.


## 4. `samtools`: Manipulating the alignment files

Now that you understand how to conduct alignments, and know what each SAM field denotes. With the `SAM` files in hand, we could call variants out of the alignment results using `samtools`.

### 4.1 Converting SAM to BAM

To achieve the goal of variant calling, our first step is to convert the SAM file to BAM. This is an important prerequisite, as all the downstream steps, including the identification of genomic variants and visualization of reads, require BAM as input.

This task, can be done using `samtools view`:
```bash
Usage:   samtools view [options] <in.bam>|<in.sam> [region1 [...]]

Options: 
         -b       output BAM
         -h       print header for the SAM output
         -H       print header only (no alignments)
         -S       input is SAM
         -u       uncompressed BAM output (force -b)
         -1       fast compression (force -b)
         -x       output FLAG in HEX (samtools-C specific)
         -X       output FLAG in string (samtools-C specific)
         -c       print only the count of matching records
         -B       collapse the backward CIGAR operation
         -@ INT   number of BAM compression threads [0]
         -L FILE  output alignments overlapping the input BED FILE [null]
         -t FILE  list of reference names and lengths (force -S) [null]
         -T FILE  reference sequence file (force -S) [null]
         -o FILE  output file name [stdout]
         -R FILE  list of read groups to be outputted [null]
         -f INT   required flag, 0 for unset [0]
         -F INT   filtering flag, 0 for unset [0]
         -q INT   minimum mapping quality [0]
         -l STR   only output reads in library STR [null]
         -r STR   only output reads in read group STR [null]
         -s FLOAT fraction of templates to subsample; integer part as seed [-1]
         -?       longer help
```

So you can run
```bash
samtools view -b -S alignments/sim_reads_mapped.sam -o alignments/sim_reads_mapped.bam
```

### <font color="red">$\S$Exercise 1</font>

1. How many of the alignments pass the mapping-quality threshold of 30? That is, the probability of mapping error is lower than $p=10^{-30/10} = 0.001$.

2. How many reads fail to map to the reference genome, which has a `FLAG` of `0x0004`? Can you list the name of these reads?

### 4.2 Sorting and Indexing

This is the help manual for `samtools sort`:
```bash
Usage:   samtools sort [options] <in.bam> <out.prefix>

Options: 
         -n        sort by read name
         -f        use <out.prefix> as full file name instead of prefix
         -o        final output to stdout
         -l INT    compression level, from 0 to 9 [-1]
         -@ INT    number of sorting and compression threads [1]
         -m INT    max memory per thread; suffix K/M/G recognized [768M]
```
and `samtools index`:
```bash
Usage: samtools index <in.bam> [out.index]
```

The next step is to sort and index the BAM file. There are two options for sorting BAM files: **by read name (-n)**, and **by genomic location (default)**. As our goal is to call genomic variants, and this requires that we "pile-up" all matching reads within a specific genomic location, we sort by location:
```bash
samtools sort alignments/sim_reads_aligned.bam alignments/sim_reads_aligned.sorted
```

Once you have sorted your BAM file, you can then index it. This enables tools, including SAMtools itself, and other genomic viewers to perform efficient random access on the BAM file, resulting in greatly improved performance. To do so, run:
```bash
samtools index alignments/sim_reads_aligned.bam
```
This will create an index file named `alignments/sim_reads_aligned.bam.bai`.

### 4.3 Identifying genomic variants

Now, we are ready to identify the genomic variants from our reads. Doing so requires two steps, and while one can easily pipe these two steps together, I have broken them out into two distinct steps below for improved clarity.

The first step is to use the SAMtools mpileup command to calculate the genotype likelihoods supported by the aligned reads in our sample.

```bash
Usage: samtools mpileup [options] in1.bam [in2.bam [...]]

Input options:

       -6           assume the quality is in the Illumina-1.3+ encoding
       -A           count anomalous read pairs
       -B           disable BAQ computation
       -b FILE      list of input BAM filenames, one per line [null]
       -C INT       parameter for adjusting mapQ; 0 to disable [0]
       -d INT       max per-BAM depth to avoid excessive memory usage [250]
       -E           recalculate extended BAQ on the fly thus ignoring existing BQs
       -f FILE      faidx indexed reference sequence file [null]
       -G FILE      exclude read groups listed in FILE [null]
       -l FILE      list of positions (chr pos) or regions (BED) [null]
       -M INT       cap mapping quality at INT [60]
       -r STR       region in which pileup is generated [null]
       -R           ignore RG tags
       -q INT       skip alignments with mapQ smaller than INT [0]
       -Q INT       skip bases with baseQ/BAQ smaller than INT [13]
       --rf INT     required flags: skip reads with mask bits unset []
       --ff INT     filter flags: skip reads with mask bits set []

Output options:

       -D           output per-sample DP in BCF (require -g/-u)
       -g           generate BCF output (genotype likelihoods)
       -O           output base positions on reads (disabled by -g/-u)
       -s           output mapping quality (disabled by -g/-u)
       -S           output per-sample strand bias P-value in BCF (require -g/-u)
       -u           generate uncompress BCF output

SNP/INDEL genotype likelihoods options (effective with `-g' or `-u'):

       -e INT       Phred-scaled gap extension seq error probability [20]
       -F FLOAT     minimum fraction of gapped reads for candidates [0.002]
       -h INT       coefficient for homopolymer errors [100]
       -I           do not perform indel calling
       -L INT       max per-sample depth for INDEL calling [250]
       -m INT       minimum gapped reads for indel candidates [1]
       -o INT       Phred-scaled gap open sequencing error probability [40]
       -p           apply -m and -F per-sample to increase sensitivity
       -P STR       comma separated list of platforms for indels [all]

Notes: Assuming diploid individuals
```

Thus we can run the following command to output the **genotype likelihood** for the possible variant sites:
```bash
samtools mpileup -g -f genomes/NC_008253.fna alignments/sim_reads_aligned.sorted.bam > variants/sim_variants.bcf
```

The `samtools mpileup` command automatically scans every position for each aligned read, computes **all the possible genotypes supported by the reads**, and then computes the **probability for each of these genotypes**.

In our case, SAMtools scans the first 1000 bases in the E. coli genome, and then weighs the collective evidence of all reads to infer the true genotype. For example, *position 42 (reference = G) has 24 reads with a G and two reads with a T (total read depth = 26)*. In this case, SAMtools concludes with high probability that the sample has a **genotype of G**, and that **T reads are likely due to sequencing errors**.

By contrast, *position 736 (reference = T) has 2 reads with a C and 66 reads with a G (total read depth = 68)*. In this case, SAMtools concludes with high probability that the sample has a genotype of G. For each position assayed, SAMtools computes all the possible genotypes, and then outputs all the results in the **binary call format (BCF)**.

The second step is to use `bcftools`:
```bash
bcftools view -c -v variants/sim_variants.bcf > variants/sim_variants.vcf
```

The `bcftools view` command uses the genotype likelihoods generated from the previous step to call SNPs and indels, and outputs the all identified variants in the **variant call format (VCF)**, which are widely used to represent genomic variants.

## 5. Understanding the VCF Format

Similar to SAM,  VCF files contain **header lines**, which begin with # or ## symbols, and **data lines**, which do not begin with a # symbol.

The header contains important information, such as the name of the program which generated the file, the VCF format version number, the reference genome name, and **information regarding individual columns within the file**.

Each data line is required 8 mandatory columns:
- **CHROM** reference, 
- **POS**ition within the chromosome, 
- **ID** of the site,
- **REF**erence allele, 
- **ALT**ernative allele, 
- **QUAL**ilities,
- **FILTER** or not,
- extra **INFO**rmation


Directly after these eight is a FORMAT column used to describe the format for all subsequent sample columns. VCF can contain information regarding multiple samples, and each sample gets its own column. 

In our example, we have only 1-sample.

We have only one variant record:
```
#CHROM  POS ID  REF ALT QUAL    FILTER  INFO    FORMAT  alignments/sim_read.sorted.bam
NC_008253.1   736  .   T   G,C   33   .    DP=76;VDB=1.369620e-01;AF1=1;AC1=2;DP4=0,0,0,69;MQ=36;FQ=-229   PL  66,202,0,68,170,62
```

Within this example VCF file, the FORMAT column is set to "PL". If you search within the header, you can find that PL is defined as:
```
##FORMAT=<ID=PL,Number=G,Type=Integer,Description="List of Phred-scaled genotype likelihoods">
```

Based on the reference and the alternative sequences, SAMtools automatically calculates the following genotypes, in this exact order:

| Genotype	| Phred Scaled Score |	Likelihood p-value |
| --- | --- | --- |
| TT |	64	| $10^{-64/10}$ = 3.981e-07 |
| TG | 178	| $10^{-178/10}$ = 2.512e-18 |
| GG | 0 | 1.0 |
| TC | 66 |	$10^{-66/10}$ = 2.512e-07 |
| GC | 147	| $10^{-147/10}$ = 1.995e-15 |
| CC | 60 |	$10^{-60/10}$ = 1.000e-06 |

[Here](https://samtools.github.io/hts-specs/VCFv4.2.pdf) is the details for a VCF file.

### <font color="red">$\S$Exercise 2: Variant Call Format (VCFs)</font>

Try your best to interpret the follwowing "*.vcf" files:
```
#CHROM    POS    ID    REF    ALT    QUAL    FILTER    INFO    FORMAT    sample1    sample2    sample3
3    74393    .    G    T     999    .    DP=31;AF1=0.7002;AC1=4;DP4=4,0,22,2    ;... GT:PL:DP:GQ    1/1:181, 57, 0:19:57    1/1:90,15,0:5:16    0/0:0,12,85:4:7
```

See H, Li. Bioinformatics, 27(21): 2987-2993 (2011) for details of likelihood and population genetics calculation.

## 6. Visualing the mapping of reads

We can use `samtools tview` for viewing the mapping results of simulated reads against the reference genome:
```bash
samtools tview alignments/sim_reads_aligned.sorted.bam genomes/NC_008253.fna
```
![](../images/tview.png)

## 7. References

- Cock PJ et al. [The Sanger FASTQ file format for sequences with quality scores, and the Solexa/Illumina FASTQ variants](https://www.ncbi.nlm.nih.gov/pubmed/20015970). Nucleic Acids Res. 2010 Apr;38(6)
- Danecek P. et al. [The variant call format and VCFtools](https://www.ncbi.nlm.nih.gov/pubmed/21653522). Bioinformatics. 2011 Aug 1;27(15):2156-8.
- Li H. et al. [The Sequence Alignment/Map format and SAMtools](https://www.ncbi.nlm.nih.gov/pubmed/19505943). Bioinformatics. 2009 Aug 15;25(16):2078-9.
- Li H. et al. [Fast and accurate short read alignment with Burrows-Wheeler transform](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2705234). Bioinformatics. 2009; 25:1754-1760.
- Li H. et al. [Fast and accurate long-read alignment with Burrows-Wheeler Transform](https://www.ncbi.nlm.nih.gov/pubmed/20080505). Bioinformatics. 2010;26:589-595.

## 8. Exercises

1. Try organizing the above process into an analysis pipeline.
2. Try a larger set of different length of paired-end reads, conduct the same analysis.
3. Try a longer reads, and compare the performance of `bwa-backtrack`, `bwa-sw` and `bwa-mem` alignment algorithms.
4. Try a real data on http://cbb.sjtu.edu.cn/course/advanced/lab2.pdf.